In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Simple Impact-T Run

In [2]:
from impact import Impact, template_dir
import os

In [3]:
ifile = os.path.join(template_dir,'lcls_injector/ImpactT.in')
os.path.exists(ifile)

True

In [4]:
# Make Impact object
I = Impact(ifile, verbose=True)

================ Impact-T Summary ================
1 bunchtotal charge: 249.99999999999997 pC
Distribution type: read
Cathode start at z = 0 m
   emission time: 1.4e-11 s
   image charges neglected after z = 0.02 m
Tracking 300004 particles
Processor domain: 1 x 1 = 1 CPUs
Computational domain: 0.015 m x 0.015 m x 45.0 m
Space charge grid: 64 x 64 x 128
Maximum time steps: 1000000
Random Seed: 6
Reference Frequency: 2856000000.0 Hz
Initial reference time: 0.0 s
Simulation starting from the beginning


Configured to run in: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmpihz8sy1y


In [5]:
# Change some things
I.input['header']['Np'] = 10000
I.input['header']['Nx'] = 16
I.input['header']['Ny'] = 16
I.input['header']['Nz'] = 16
I.input['header']['Dt'] = 5e-13

# Other switches
#I.timeout = 1000
# Switches for MPI
I.use_mpi=True
I.input['header']['Npcol'] = 4
I.impact_bin = '$IMPACTT_MPI_BIN'

In [6]:
# Change stop location
I.ele['stop_1']['s'] = 2.0
#I.ele['stop_1']['s'] = I.ele['OTR2']['s']+.001

In [7]:
I.run()

Running Impact-T in /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmpihz8sy1y
 !-----------------------------------------------------------
 ! IMPACT-T Parallel Beam Dynamics Tracking Code: 2.0 beta version
 ! Copyright of The Regents of the University of California
 !-----------------------------------------------------------
 nblem:           57          57
 pass setting up lattice...
 check randomness:            0  0.79479214134216858     
 check randomness:            3  0.79369999429767613     
 check randomness:            1  0.65948074101936072     
 check randomness:            2  0.68444898109486740     
 avgpts, jlow, and jhigh:         2500           1        2500
 avgpts, jlow, and jhigh:         2500        5001        7500
 avgpts, jlow, and jhigh:         2500        7501       10000
 avgpts, jlow, and jhigh:         2500        2501        5000
 time:    59.798043000000000, elapsed: 60.488285779953Finished.
Loaded fort 30 : Fourth root of the fourth moments of the b

In [8]:
I.particles.keys()

dict_keys(['initial_particles', 'final_particles', 'WRITE_BEAM_41_BEGL0'])

In [9]:
I.particles.keys()

dict_keys(['initial_particles', 'final_particles', 'WRITE_BEAM_41_BEGL0'])

In [10]:
I.output['stats'].keys()

dict_keys(['t', 'z', 'x_moment4', 'GBx_moment4', 'y_moment4', 'GBy_moment4', 'z_moment4', 'GBz_moment4', 'y_centroid', 'y_rms', 'GBy_centroid', 'GBy_rms', 'y_twiss', 'y_normemit', 'x_centroid', 'x_rms', 'GBx_centroid', 'GBx_rms', 'x_twiss', 'x_normemit', 'z_centroid', 'z_rms', 'GBz_centroid', 'GBz_rms', 'z_twiss', 'z_normemit', 'numparticles_min', 'numparticles_max', 'numparticles', 'x_moment3', 'GBx_moment3', 'y_moment3', 'GBy_moment3', 'z_moment3', 'GBz_moment3', 'gamma', 'E_kinetic', 'beta', 'r_max', 'deltaGamma_rms', 'x_max', 'GBx_max', 'y_max', 'GBy_max', 'z_max', 'GBz_max'])

In [11]:
I.output['slice_info']

{'final_particle_slices': {'slice_z': array([-1.164448e-03, -1.009188e-03, -8.539284e-04, -6.986687e-04,
         -5.434090e-04, -3.881493e-04, -2.328896e-04, -7.762985e-05,
          7.762985e-05,  2.328896e-04,  3.881493e-04,  5.434090e-04,
          6.986687e-04,  8.539284e-04,  1.009188e-03,  1.164448e-03]),
  'particles_per_cell': array([  24.44682,  164.8737 ,  389.3622 ,  560.9102 ,  748.6435 ,
          835.2737 ,  939.2221 ,  990.673  , 1026.228  ,  969.5162 ,
          936.008  ,  832.2522 ,  694.4217 ,  536.9188 ,  299.5515 ,
           51.69837]),
  'current': array([ 1.180113,  7.958895, 18.79555 , 27.07663 , 36.13902 , 40.32089 ,
         45.33876 , 47.82243 , 49.53876 , 46.80113 , 45.1836  , 40.17503 ,
         33.52159 , 25.9185  , 14.46015 ,  2.495618]),
  'x_emittance': array([2.035768e-07, 4.311252e-07, 6.483179e-07, 8.144251e-07,
         1.001361e-06, 9.434810e-07, 8.404113e-07, 7.222086e-07,
         7.320311e-07, 7.974904e-07, 8.696340e-07, 9.823986e-07,
        

# Plot

In [12]:
from bokeh.plotting import output_notebook
from bokeh.plotting import show
output_notebook(hide_banner=True)

from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Range1d
from impact.parsers import load_many_fort
from impact.parsers import UNITS

from impact.plotting import layout_plot


ds = ColumnDataSource(I.output['stats'])

In [13]:
# Change somethone
I.ele['QE01']['b1_gradient'] = 0
layout = layout_plot(I.input['lattice'], plot_height=500)
show(layout)

In [14]:
def plot1(y_axis, x_axis='z', source=ds):
    yunit = UNITS[y_axis]
    ylabel = y_axis+' ('+yunit+')'
    
    xunit = UNITS[x_axis]
    xlabel = x_axis+' ('+xunit+')'
    p = figure(plot_width=600, plot_height=200, x_axis_label=xlabel, y_axis_label=ylabel)
    p.line( x_axis, y_axis, source=source, color='red')
    return p
show(plot1('gamma'))

# Archiving

In [15]:
from impact import writers
import h5py

In [16]:
# This is the total bunch charge in Coulombs
total_charge = I.input['header']['Bcurr'] / I.input['header']['Bfreq']
total_charge

2.4999999999999996e-10

In [17]:
I.particles

{'initial_particles': array([( 4.3611631e-04,  1.2615826e-03,  3.2937175e-04, -5.7131252e-04, -2.7340319e-06, 0.001978),
        ( 4.0614893e-04,  1.0219854e-03,  3.8438809e-04,  7.9314477e-04, -1.7480634e-06, 0.001978),
        (-4.6228725e-06, -9.2105950e-05, -1.0653409e-04, -3.6416778e-05, -2.1070141e-06, 0.001978),
        ...,
        ( 1.9058713e-04, -1.3542358e-03,  6.4437180e-06, -9.8451886e-06, -2.3828392e-06, 0.001978),
        (-3.7396643e-04,  1.4488924e-03, -2.0164375e-04,  8.9741137e-04, -1.9571324e-06, 0.001978),
        (-3.8447387e-04, -1.3964400e-04,  1.0404854e-04, -4.2386647e-04, -1.3655324e-06, 0.001978)],
       dtype=[('x', '<f8'), ('GBx', '<f8'), ('y', '<f8'), ('GBy', '<f8'), ('z', '<f8'), ('GBz', '<f8')]),
 'final_particles': array([( 1.76203264e-04, -0.0017843 , -0.00033034, 0.00278289, 2.00024475, 29.86484765),
        ( 1.21189572e-04, -0.0052989 , -0.00017832, 0.0029838 , 2.00006115, 29.82126512),
        ( 2.98053190e-04, -0.00126499, -0.00021247, 0.001870

In [18]:
h5 = h5py.File('test.h5', 'w')

In [19]:
I.archive(h5)

Archiving initial_particles with charge 2.4997499999999996e-10
Archiving final_particles with charge 2.4997499999999996e-10
Archiving WRITE_BEAM_41_BEGL0 with charge 2.4997499999999996e-10


In [20]:
h5.close()

# Reading archive

In [21]:
# Reading input
from impact.readers import read_input_h5

In [22]:
h5 = h5py.File('test.h5', 'r')

In [23]:
input = read_input_h5(h5['input/'])
list(input)

['header', 'lattice', 'input_particle_file', 'fieldmaps']

In [24]:
list(input['fieldmaps'])

['rfdata102', 'rfdata201', 'rfdata4', 'rfdata5', 'rfdata6', 'rfdata7']

# Read and Plot Particles

In [25]:
from bokeh.plotting import  show, output_notebook
from bokeh.layouts import column, row
output_notebook(verbose=False, hide_banner=True)
import os

from opmd_beamphysics import bunch_plotting, bunch_tools, bunch_stats

In [26]:
list(h5)

['input', 'particles', 'run_info', 'slice_info', 'stats']

In [27]:
list(h5['particles'])

['WRITE_BEAM_41_BEGL0', 'final_particles', 'initial_particles']

In [28]:
bunch1 = h5['particles/WRITE_BEAM_41_BEGL0']
show(
    column(
        
    bunch_plotting.plot_bunch_h5(bunch1, 'z', 'x', bins = 50, liveOnly=False),
    bunch_plotting.plot_histogram_h5(bunch1, 'z', bins=100,liveOnly=False),
    bunch_plotting.plot_bunch_h5(bunch1, 'z', 'pz', bins = 50, liveOnly=False),
   # bunch_plotting.plot_bunch_current_profile(bunch1, bins=100, liveOnly=False)
    
    
    ))

In [29]:
# Cleanup
os.remove('test.h5')